In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import csv
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
credits = pd.read_csv("credit.csv")

In [ ]:
credits.columns

In [ ]:
credits.info()

In [ ]:
type(credits['DISSEMINATION_ID'].tolist()[0])

In [ ]:
credits["DISSEMINATION_ID"] = credits["DISSEMINATION_ID"].convert_objects(convert_numeric=True)

In [ ]:
credits["ORIGINAL_DISSEMINATION_ID"] = credits["ORIGINAL_DISSEMINATION_ID"].convert_objects(convert_numeric=True)

In [ ]:
#credits.loc[credits["ORIGINAL_DISSEMINATION_ID"] == 61975011]

In [ ]:
credits_new = credits[["DISSEMINATION_ID","ORIGINAL_DISSEMINATION_ID","ACTION","EXECUTION_TIMESTAMP","CLEARED",'EFFECTIVE_DATE', 'END_DATE', 'DAY_COUNT_CONVENTION','SETTLEMENT_CURRENCY','TAXONOMY','PRICE_NOTATION_TYPE', 'PRICE_NOTATION','ROUNDED_NOTIONAL_AMOUNT_1']]

In [ ]:
credits_new = credits_new[credits_new["CLEARED"] == "C"]
# Filter out NaN from PRICE_NOTATION_TYPE
credits_new = credits_new[credits_new["PRICE_NOTATION_TYPE"].isnull() == False] # 1% null value
credits_new

In [ ]:
credits.info()

In [ ]:
credits_new["TAXONOMY"].unique()

In [ ]:
# Filter for USD Taxonomy HY and IG only
mask = (credits_new["TAXONOMY"] == 'Credit:Index:CDX:CDXIG') | (credits_new["TAXONOMY"]=='Credit:Index:CDX:CDXHY')
credits_new = credits_new[mask]


In [ ]:
# Resetting INDEX
credits_new = credits_new.reset_index(drop = True)
credits_new

In [ ]:
credits_new["TAXONOMY"].unique()

In [ ]:
#pd.to_datetime(credits_new["EFFECTIVE_DATE"],format="%Y-%m-%d")
#pd.to_datetime(credits_new['END_DATE'],format="%Y-%m-%d")

# Abnormal Effective Date
abnormal_date_entry = credits_new[credits_new["EFFECTIVE_DATE"] == "2571-10-09"]

In [ ]:
credits_new = credits_new.drop(abnormal_date_entry.index)
credits_new = credits_new.reset_index(drop = True)
credits_new

In [ ]:
# Date Time Format Conversion
credits_new["EFFECTIVE_DATE"]= pd.to_datetime(credits_new["EFFECTIVE_DATE"])
credits_new['END_DATE']=pd.to_datetime(credits_new['END_DATE'])
credits_new.info()

In [ ]:
credits_new["SETTLEMENT_CURRENCY"].value_counts()
#credits_new["SETTLEMENT_CURRENCY"].count()

In [ ]:
# Fill N/A  in Currency with USD
credits_new["SETTLEMENT_CURRENCY"] = credits_new["SETTLEMENT_CURRENCY"].fillna("USD")

In [ ]:
credits_new["SETTLEMENT_CURRENCY"].value_counts()

In [ ]:
# Filter for valid time  4.5-5.5 years
credits_new2 = credits_new.loc[(credits_new["END_DATE"]- credits_new["EFFECTIVE_DATE"] >= dt.timedelta(days = 1643))  & (credits_new["END_DATE"] - credits_new["EFFECTIVE_DATE"] <= dt.timedelta(days = 2007))] 
credits_new2

In [ ]:
# More time stamp conversion
credits_new2["EXECUTION_TIMESTAMP"]= pd.to_datetime(credits_new2["EXECUTION_TIMESTAMP"])

In [ ]:
# Reset Index again
credits_new2 = credits_new2.reset_index(drop = True)
credits_new2

In [ ]:
credits_new2["DAY_COUNT_CONVENTION"].unique()

In [ ]:
credits_new2["DAY_COUNT_CONVENTION"].value_counts()

In [ ]:
# Filter for Non "ACT/360" Entries
credits_new2 = credits_new2.loc[credits_new2["DAY_COUNT_CONVENTION"] == ' ACT/360']

In [ ]:
credits_new2["PRICE_NOTATION_TYPE"].unique()

In [ ]:
# Remap similar entry names
credits_new2["PRICE_NOTATION_TYPE"] = credits_new2["PRICE_NOTATION_TYPE"].map({"BasisPoints":"Basis points", "Basis Points": "Basis points", "Basispoints":"Basis points", "UpfrontPoints": "UpfrontPoints", "Price":"Price", "Percentage":"Percentage"})

In [ ]:
credits_new2.head()

In [ ]:
# Encoding for + in Notional Amount 
l = credits_new2["ROUNDED_NOTIONAL_AMOUNT_1"].tolist()
encoding = [1 if '+' in n else 0 for n in l]
credits_new2["EXCEEDS_ROUNDED_NOTIONAL_AMOUNT"] = encoding

# Fix Price Notation/ROUNDED_NOTIONAL_AMOUNT_1 Formatting
credits_new2 = credits_new2.dropna(axis = 0,subset = ["PRICE_NOTATION"])
credits_new2["PRICE_NOTATION"] = credits_new2["PRICE_NOTATION"].replace(',','')
credits_new2["ROUNDED_NOTIONAL_AMOUNT_1"] = credits_new2["ROUNDED_NOTIONAL_AMOUNT_1"].str.replace(",","").str.replace("+","")
credits_new2["PRICE_NOTATION"] = credits_new2["PRICE_NOTATION"].convert_objects(convert_dates='coerce', convert_numeric=True)

In [ ]:
credits_new2['PRICE_NOTATION'].hist(by=credits_new2['TAXONOMY'],bins = 1000)

In [ ]:
#credits_new2 = credits_new2.drop(["SETTLEMENT_CURRENCY","CLEARED"],axis=1)

In [ ]:
# Numeric format conversion
#credits_new2["ORIGINAL_DISSEMINATION_ID"] = pd.to_numeric(credits_new2["ORIGINAL_DISSEMINATION_ID"])

In [ ]:
#credits_new2['DISSEMINATION_ID'] = credits_new2.DISSEMINATION_ID.astype(int)

In [ ]:
# Remove all the 47 upfrontpoints: 
credits_new2 = credits_new2.loc[credits_new2["PRICE_NOTATION_TYPE"] != "UpfrontPoints"]
credits_new2

In [ ]:
# Create a new column of total value of the orders:
price_type = credits_new2["PRICE_NOTATION_TYPE"].unique().tolist()

In [ ]:
credits_new2["ROUNDED_NOTIONAL_AMOUNT_1"] = credits_new2["ROUNDED_NOTIONAL_AMOUNT_1"].astype(float)

In [ ]:
total_amount = []
for i in range(credits_new2.shape[0]):
    if credits_new2.iloc[i]["PRICE_NOTATION_TYPE"] == "Basis points":
        total_amount.append(credits_new2.iloc[i]["PRICE_NOTATION"] * (1/10000) * credits_new2.iloc[i]["ROUNDED_NOTIONAL_AMOUNT_1"])
    elif credits_new2.iloc[i]["PRICE_NOTATION_TYPE"] == "Price":
        total_amount.append(credits_new2.iloc[i]["PRICE_NOTATION"] * credits_new2.iloc[i]["ROUNDED_NOTIONAL_AMOUNT_1"])
    else:
        total_amount.append(credits_new2.iloc[i]["PRICE_NOTATION"] * 0.01 * credits_new2.iloc[i]["ROUNDED_NOTIONAL_AMOUNT_1"])
total_amount        

In [ ]:
credits_new2["Payment"] = total_amount 
print (min(total_amount))
print(max(total_amount))

In [ ]:
credits_new2["Payment"].hist(range=[0,1000000])

In [ ]:
#plt.boxplot(credits_new2["Payment"][1:int(len(total_amount)/10)])

In [ ]:
# find out price = 0 observation
price0 = credits_new2.loc[credits_new2["PRICE_NOTATION"]==0]
price0

In [ ]:
price0['ACTION'].value_counts()

In [ ]:
credits_new2.to_csv("credits_clean_v2.csv",index = False)

In [ ]:
# Find all Canceld / Correct ACTION assuming only these have non-null ORIGINAL_DISSEMINATION_ID
changed = credits_new2.loc[credits_new2["ORIGINAL_DISSEMINATION_ID"].isnull() == False ]
changed

In [ ]:
# Find mismatched entries (Orphan entry where there is only one of Cancel or Correct)
# Because when an entry is corrected, future corrections/cancelations will use the newly generated DISSEMINATION_ID
group_1 = changed.groupby('ORIGINAL_DISSEMINATION_ID').count()


In [ ]:
changed.sort_values(by =['ORIGINAL_DISSEMINATION_ID'])

In [ ]:
finalListOfId = list(map(int, credits_new2["DISSEMINATION_ID"]))
originalListOfId =  credits["DISSEMINATION_ID"]
print("originalLen: " + str(len(set(originalListOfId))))
print("finalLen: " + str(len(set(finalListOfId))))
eliminated = list(set(originalListOfId) - set(finalListOfId))
print("eliminated: " + str(len(eliminated)))

In [ ]:
# Remove all entries considered mismatched (orphan entries)        
credits_orphan_eliminated = credits_new2[~credits_new2["ORIGINAL_DISSEMINATION_ID"].isin(eliminated)]
credits_orphan_eliminated

In [ ]:
# Find all NEW Cancel/Correct ACTION entries after orphans have been removed
changed2 = credits_orphan_eliminated.loc[credits_orphan_eliminated["ORIGINAL_DISSEMINATION_ID"].isnull() == False ]
changed2['ACTION'].value_counts() # find 174 NEW action but has original_id


In [ ]:
#Store abnormal New Action:
abnormal_New = changed2.loc[changed2["ACTION"] == "NEW"]
abnormal_New["ACTION"].value_counts()

#removed abnormal new:
changed2 = changed2.loc[changed2["ACTION"] != "NEW"]

In [ ]:
# Print changed2 in matching pairs
changed2.sort_values(by =['ORIGINAL_DISSEMINATION_ID',"ACTION", "EXECUTION_TIMESTAMP"])

In [ ]:
changed2['ACTION'].value_counts()

In [ ]:
# percentage of correct in total observations: 2.4%
# percentage of cancel in total observations: 1.6%

In [ ]:
cancel_correct = changed2.groupby('ORIGINAL_DISSEMINATION_ID').count()
cancel_correct.head()

In [ ]:
cancel_correct['DISSEMINATION_ID'].unique()

In [ ]:
# All entries where we see ORIGINAL_DISSEMINATION_ID exactly twice (aka. Cancel/Correct pair)
correct = cancel_correct.loc[cancel_correct["DISSEMINATION_ID"] == 2].index.tolist()

In [ ]:
# All entries where we see ORIGINAL_DISSEMINATION_ID exactly once (aka. Cancel action)
cancel = cancel_correct.loc[cancel_correct["DISSEMINATION_ID"] == 1].index.tolist()

In [ ]:
# All Cancel entries
df_cancel = credits_orphan_eliminated[credits_orphan_eliminated["ORIGINAL_DISSEMINATION_ID"].isin(cancel)]

In [ ]:
df_cancel = df_cancel.sort_values(by =['ORIGINAL_DISSEMINATION_ID',"EXECUTION_TIMESTAMP"])                           
df_cancel

In [ ]:
df_cancel["ACTION"].value_counts() # 4 correct in df.cancel, maybe because of the dropping null and upfront price in price type
# we found in the original credits csv, but we can't find the new action 

In [ ]:
df_cancel["TAXONOMY"].value_counts().plot.bar()

In [ ]:
df_cancel["Payment"].hist()

In [ ]:
df_cancel["PRICE_NOTATION"].hist()

In [ ]:
df_cancel["ROUNDED_NOTIONAL_AMOUNT_1"].hist()

In [ ]:
df_cancel["EXCEEDS_ROUNDED_NOTIONAL_AMOUNT"].hist()

In [ ]:
#Create a dataframe of all corrected entries:
df_correct = credits_orphan_eliminated[credits_orphan_eliminated["ORIGINAL_DISSEMINATION_ID"].isin(correct)]

In [ ]:
#sort the corrected dataframe:
df_correct = df_correct.sort_values(by =['ORIGINAL_DISSEMINATION_ID',"EXECUTION_TIMESTAMP","ACTION"])

In [ ]:
df_correct["ACTION"].value_counts()

In [ ]:
df_correct["TAXONOMY"].value_counts().plot.bar()

In [ ]:
df_correct["Payment"].hist()

In [ ]:
df_correct["PRICE_NOTATION"].hist()

In [ ]:
df_correct["ROUNDED_NOTIONAL_AMOUNT_1"].hist()

In [ ]:
df_correct["EXCEEDS_ROUNDED_NOTIONAL_AMOUNT"].hist()

In [ ]:
to_find_new = df_correct["ORIGINAL_DISSEMINATION_ID"].tolist()

In [ ]:
df_correct_NEW = credits_orphan_eliminated[credits_orphan_eliminated["DISSEMINATION_ID"].isin(to_find_new)]

In [ ]:
df_correct_NEW.sort_values(by =["DISSEMINATION_ID"])